In [234]:
import numpy as np
import pandas as pd

In [235]:
df = pd.read_csv('tmdb-movies.csv')
df.head(2)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08


In [236]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  str    
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  str    
 6   cast                  10790 non-null  str    
 7   homepage              2936 non-null   str    
 8   director              10822 non-null  str    
 9   tagline               8042 non-null   str    
 10  keywords              9373 non-null   str    
 11  overview              10862 non-null  str    
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  str    
 14  production_companies  9836 non-null   str    
 15  release_date          10866 no

In [237]:
movies = df[['imdb_id', 'original_title', 'cast', 'director', 'keywords', 'overview', 'genres', 'release_year', 'vote_average']]

In [238]:
movies.isnull().sum()

imdb_id             10
original_title       0
cast                76
director            44
keywords          1493
overview             4
genres              23
release_year         0
vote_average         0
dtype: int64

In [239]:
movies.dropna(inplace=True, ignore_index=True)

In [240]:
movies.duplicated().sum()

np.int64(1)

In [241]:
movies.drop_duplicates(inplace=True, ignore_index=True)

In [242]:
movies.info()

<class 'pandas.DataFrame'>
RangeIndex: 9302 entries, 0 to 9301
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_id         9302 non-null   str    
 1   original_title  9302 non-null   str    
 2   cast            9302 non-null   str    
 3   director        9302 non-null   str    
 4   keywords        9302 non-null   str    
 5   overview        9302 non-null   str    
 6   genres          9302 non-null   str    
 7   release_year    9302 non-null   int64  
 8   vote_average    9302 non-null   float64
dtypes: float64(1), int64(1), str(7)
memory usage: 654.2 KB


In [243]:
def convert(s):
    L = []
    for c in s.split("|"):
        L.append(c.replace(" ", "").lower())
    return L[:3]


In [244]:
movies['cast'] = movies['cast'].apply(convert)

In [245]:
movies['genres'] = movies['genres'].apply(convert)

In [246]:
movies['keywords'] = movies['keywords'].apply(convert)

In [247]:
movies['director'] = movies['director'].apply(lambda x: [x.replace(" ", "").lower()])

In [248]:
movies['overview'] = movies['overview'].apply(lambda x: x.lower().split(" "))

In [249]:
movies['tags'] = movies['overview'] + 2*movies['keywords'] + 2*movies['genres'] + movies['cast'] + 2*movies['director']

In [250]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [251]:
movies_df = movies[['imdb_id', 'original_title', 'release_year', 'vote_average', 'tags']]

In [252]:
movies_df.head(1)

,imdb_id,original_title,release_year,vote_average,tags
0,tt0369610,Jurassic World,2015,6.5,twenty-two years after the events of jurassic ...


In [253]:
posters = []
def get_poster():
    MOVIE = pd.read_csv('MOVIE.csv')
    for i in movies_df['imdb_id']:
        poster = MOVIE[MOVIE['IMDB_ID'] == i]['POSTER_PATH']
        if len(poster):
            posters.append(poster.values[0])
        else:
            posters.append(None)
get_poster()

In [254]:
movies_df['poster'] = posters

In [255]:
movies_df

,imdb_id,original_title,release_year,vote_average,tags,poster
0,tt0369610,Jurassic World,2015,6.5,twenty-two years after the events of jurassic ...,/rhr4y79GpxQF9IsfJItRXVaoGs4.jpg
1,tt1392190,Mad Max: Fury Road,2015,7.1,an apocalyptic story set in the furthest reach...,/8tZYtuWezp8JbcsvHYO0O46tFbo.jpg
2,tt2908446,Insurgent,2015,6.3,beatrice prior must confront her inner demons ...,/dP5Fb6YRfzmCQtRbHOr2kO7tJW9.jpg
3,tt2488496,Star Wars: The Force Awakens,2015,7.5,thirty years after defeating the galactic empi...,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg
4,tt2820852,Furious 7,2015,7.3,deckard shaw seeks revenge against dominic tor...,/jPFsjmFlTFmpIY41BP3C4sVYOKw.jpg
...,...,...,...,...,...,...
9297,tt0060371,The Endless Summer,1966,7.4,"the endless summer, by bruce brown, is one of ...",/5W5uaqQ7NZTwoDMKO4AtdcahHex.jpg
9298,tt0060472,Grand Prix,1966,5.7,grand prix driver pete aron is fired by his te...,/yEXrf09o8wVRFSptYFr6VcqLjPr.jpg
9299,tt0060161,Beregis Avtomobilya,1966,6.5,an insurance agent who moonlights as a carthie...,NaN
9300,tt0061177,"What's Up, Tiger Lily?",1966,5.4,"in comic woody allen's film debut, he took the...",/z3ABDEfhLqX09nXgp6ilKW44jAB.jpg


In [256]:
movies_df.isnull().sum()

imdb_id             0
original_title      0
release_year        0
vote_average        0
tags                0
poster            538
dtype: int64

In [257]:
movies_df.dropna(inplace=True, ignore_index=True)

In [258]:
movies_df


,imdb_id,original_title,release_year,vote_average,tags,poster
0,tt0369610,Jurassic World,2015,6.5,twenty-two years after the events of jurassic ...,/rhr4y79GpxQF9IsfJItRXVaoGs4.jpg
1,tt1392190,Mad Max: Fury Road,2015,7.1,an apocalyptic story set in the furthest reach...,/8tZYtuWezp8JbcsvHYO0O46tFbo.jpg
2,tt2908446,Insurgent,2015,6.3,beatrice prior must confront her inner demons ...,/dP5Fb6YRfzmCQtRbHOr2kO7tJW9.jpg
3,tt2488496,Star Wars: The Force Awakens,2015,7.5,thirty years after defeating the galactic empi...,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg
4,tt2820852,Furious 7,2015,7.3,deckard shaw seeks revenge against dominic tor...,/jPFsjmFlTFmpIY41BP3C4sVYOKw.jpg
...,...,...,...,...,...,...
8759,tt0060955,Seconds,1966,6.6,a secret organisation offers wealthy people a ...,/mZtEXN7b7DWsn0BEqIKB10eZFi9.jpg
8760,tt0060214,Carry On Screaming!,1966,7.0,the sinister dr watt has an evil scheme going....,/A9fAE8hZoCyrdgFGzjXSuT92mUe.jpg
8761,tt0060371,The Endless Summer,1966,7.4,"the endless summer, by bruce brown, is one of ...",/5W5uaqQ7NZTwoDMKO4AtdcahHex.jpg
8762,tt0060472,Grand Prix,1966,5.7,grand prix driver pete aron is fired by his te...,/yEXrf09o8wVRFSptYFr6VcqLjPr.jpg


In [259]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=10000,
    stop_words='english',
    ngram_range=(1,2)
)

vectors = tfidf.fit_transform(movies_df['tags'])

In [260]:
from sklearn.metrics.pairwise import cosine_similarity
def recommend(imdb_id):
    idx = movies_df[movies_df['imdb_id'] == imdb_id].index[0]
    sim = cosine_similarity(vectors[idx], vectors)
    top = sorted(list(enumerate(sim[0])), key=lambda x: x[1], reverse=True)[1:11]
    top_movies = []
    for i in top:
        m = movies_df.iloc[i[0]]
        top_movies.append(m.to_dict())
    return top_movies


In [261]:
imdb = movies_df[movies_df['original_title'] == 'Spider-Man']['imdb_id'].values[0]
recommend(imdb)

[{'imdb_id': 'tt0413300',
  'original_title': 'Spider-Man 3',
  'release_year': 2007,
  'vote_average': 5.8,
  'tags': "the seemingly invincible spider-man goes up against an all-new crop of villains -- including the shape-shifting sandman -- in the third installment of this blockbusting comic book adventure series. while spidey's superpowers are altered by an alien organism, his alter ego, peter parker, deals with nemesis eddie brock and gets caught up in a love triangle. dualidentity amnesia sandstorm dualidentity amnesia sandstorm fantasy action adventure fantasy action adventure tobeymaguire kirstendunst jamesfranco samraimi samraimi",
  'poster': '/2jLxKF73SAPkyhIWrnv67IH7kJ1.jpg'},
 {'imdb_id': 'tt0316654',
  'original_title': 'Spider-Man 2',
  'release_year': 2004,
  'vote_average': 6.4,
  'tags': "peter parker is going through a major identity crisis. burned out from being spider-man, he decides to shelve his superhero alter ego, which leaves the city suffering in the wake of c

In [262]:
imdb = movies_df[movies_df['original_title'] == 'Se7en']['imdb_id'].values[0]
recommend(imdb)

[{'imdb_id': 'tt2236054',
  'original_title': 'Mystery Road',
  'release_year': 2013,
  'vote_average': 6.1,
  'tags': 'an indigenous detective returns to the outback to investigate the murder of a young girl. detective australia sniperrifle detective australia sniperrifle thriller crime mystery thriller crime mystery aaronpedersen hugoweaving jackthompson ivansen ivansen',
  'poster': '/vmcfuVKkXLwQqW6E9mq9wdUWyaZ.jpg'},
 {'imdb_id': 'tt0278504',
  'original_title': 'Insomnia',
  'release_year': 2002,
  'vote_average': 6.6,
  'tags': "two los angeles homicide detectives are dispatched to a northern town where the sun doesn't set to investigate the methodical murder of a local teen. detective confession fbi detective confession fbi crime mystery thriller crime mystery thriller alpacino robinwilliams mauratierney christophernolan christophernolan",
  'poster': '/cwB0t4OHX1Pw1Umzc9jPgzalUpS.jpg'},
 {'imdb_id': 'tt0443706',
  'original_title': 'Zodiac',
  'release_year': 2007,
  'vote_ave

In [263]:
imdb = movies_df[movies_df['original_title'] == "Harry Potter and the Philosopher's Stone"]['imdb_id'].values[0]
recommend(imdb)

[{'imdb_id': 'tt0295297',
  'original_title': 'Harry Potter and the Chamber of Secrets',
  'release_year': 2002,
  'vote_average': 7.2,
  'tags': 'ignoring threats to his life, harry returns to hogwarts to investigate â€“ aided by ron and hermione â€“ a mysterious series of attacks. flyingcar witch magiclanguage flyingcar witch magiclanguage adventure fantasy family adventure fantasy family danielradcliffe rupertgrint emmawatson chriscolumbus chriscolumbus',
  'poster': '/sdEOH0992YZ0QSxgXNIGLq1ToUi.jpg'},
 {'imdb_id': 'tt0417741',
  'original_title': 'Harry Potter and the Half-Blood Prince',
  'release_year': 2009,
  'vote_average': 7.3,
  'tags': "as harry begins his sixth year at hogwarts, he discovers an old book marked as 'property of the half-blood prince', and begins to learn more about lord voldemort's dark past. witch broom schoolofwitchcraft witch broom schoolofwitchcraft adventure fantasy family adventure fantasy family danielradcliffe rupertgrint emmawatson davidyates david

In [264]:
imdb = movies_df[movies_df['original_title'] == "The Avengers"]['imdb_id'].values[0]
recommend(imdb)

[{'imdb_id': 'tt2395427',
  'original_title': 'Avengers: Age of Ultron',
  'release_year': 2015,
  'vote_average': 7.4,
  'tags': 'when tony stark tries to jumpstart a dormant peacekeeping program, things go awry and earthâ€™s mightiest heroes are put to the ultimate test as the fate of the planet hangs in the balance. as the villainous ultron emerges, it is up to the avengers to stop him from enacting his terrible plans, and soon uneasy alliances and unexpected action pave the way for an epic and unique global adventure. marvelcomic comic sequel marvelcomic comic sequel action adventure sciencefiction action adventure sciencefiction robertdowneyjr. chrishemsworth markruffalo josswhedon josswhedon',
  'poster': '/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg'},
 {'imdb_id': 'tt2011109',
  'original_title': "Marvel One-Shot: A Funny Thing Happened on the Way to Thor's Hammer",
  'release_year': 2011,
  'vote_average': 7.1,
  'tags': 'agent coulson stops at a convenience store and deals with a coincid

In [265]:
imdb = movies_df[movies_df['original_title'] == "Fight Club"]['imdb_id'].values[0]
recommend(imdb)

[{'imdb_id': 'tt0421715',
  'original_title': 'The Curious Case of Benjamin Button',
  'release_year': 2008,
  'vote_average': 7.2,
  'tags': 'tells the story of benjamin button, a man who starts aging backwards with bizarre consequences. diary navy funeral diary navy funeral fantasy drama thriller fantasy drama thriller cateblanchett bradpitt tildaswinton davidfincher davidfincher',
  'poster': '/26wEWZYt6yJkwRVkjcbwJEFh9IS.jpg'},
 {'imdb_id': 'tt3442006',
  'original_title': 'Cake',
  'release_year': 2014,
  'vote_average': 6.1,
  'tags': 'after having visions of a member of her support group who killed herself, a woman who also suffers with chronic pain seeks out the widower of the suicide. suicide supportgroup depression suicide supportgroup depression drama drama jenniferaniston annakendrick williamh.macy danielbarnz danielbarnz',
  'poster': '/qx6BB96iG1kN3ITL7kBfe3L2l2c.jpg'},
 {'imdb_id': 'tt2267998',
  'original_title': 'Gone Girl',
  'release_year': 2014,
  'vote_average': 7.

In [266]:
import pickle
pickle.dump(movies_df, open('../backend/movies.pkl', 'wb'))